In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin
import random
from spacy.util import minibatch,compounding
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import KFold
import itertools


In [ ]:

def load_data(file_path):
  with open(file_path,'r',encoding='utf-8') as f:
    lines=f.readlines()
  data=[]
  sentence=[]
  labels=[]
  for line in lines:
    if line.strip()=="":
      if sentence:
        data.append((sentence,labels))
        sentence=[]
        labels=[]
    else:
      parts = line.strip().split()
      if len(parts)>=4:
        word,label=parts[0],parts[-1]
        sentence.append(word)
        labels.append(label)
  return data
train_data=load_data('eng.train')
testa_data=load_data('eng.testa')
testb_data=load_data('eng.testb')

print(len(train_data))
train_data[:2]

14987


[(['-DOCSTART-'], ['O']),
 (['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'],
  ['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O'])]

In [ ]:

word_to_ix={'<UNK>':0}
tag_to_ix={}
for sentence,tags in train_data:
  for word in sentence:
    if word not in word_to_ix:
      word_to_ix[word]=len(word_to_ix)
  for tag in tags:
    if tag not in tag_to_ix:
      tag_to_ix[tag]=len(tag_to_ix)
print(word_to_ix)
print(tag_to_ix)

{'<UNK>': 0, '-DOCSTART-': 1, 'EU': 2, 'rejects': 3, 'German': 4, 'call': 5, 'to': 6, 'boycott': 7, 'British': 8, 'lamb': 9, '.': 10, 'Peter': 11, 'Blackburn': 12, 'BRUSSELS': 13, '1996-08-22': 14, 'The': 15, 'European': 16, 'Commission': 17, 'said': 18, 'on': 19, 'Thursday': 20, 'it': 21, 'disagreed': 22, 'with': 23, 'advice': 24, 'consumers': 25, 'shun': 26, 'until': 27, 'scientists': 28, 'determine': 29, 'whether': 30, 'mad': 31, 'cow': 32, 'disease': 33, 'can': 34, 'be': 35, 'transmitted': 36, 'sheep': 37, 'Germany': 38, "'s": 39, 'representative': 40, 'the': 41, 'Union': 42, 'veterinary': 43, 'committee': 44, 'Werner': 45, 'Zwingmann': 46, 'Wednesday': 47, 'should': 48, 'buy': 49, 'sheepmeat': 50, 'from': 51, 'countries': 52, 'other': 53, 'than': 54, 'Britain': 55, 'scientific': 56, 'was': 57, 'clearer': 58, '"': 59, 'We': 60, 'do': 61, "n't": 62, 'support': 63, 'any': 64, 'such': 65, 'recommendation': 66, 'because': 67, 'we': 68, 'see': 69, 'grounds': 70, 'for': 71, ',': 72, 'chi

In [ ]:

def prepare_sequence(seq,to_ix):
  idxs=[to_ix[w] if w in to_ix else to_ix['<UNK>'] for w in seq]
  return torch.tensor(idxs,dtype=torch.long)

In [ ]:

class NERDataset(Dataset):
  def __init__(self,data,word_to_ix,tag_to_ix):
    self.data=data
    self.word_to_ix=word_to_ix
    self.tag_to_ix=tag_to_ix
  def __len__(self):
    return len(self.data)
  def __getitem__(self,idx):
    sentence,tags=self.data[idx]
    sentence_in=prepare_sequence(sentence,self.word_to_ix)
    tags_in=prepare_sequence(tags,self.tag_to_ix)
    return sentence_in,tags_in
def collate_fn(batch):
  sentences,tags=zip(*batch)
  sentences_padded=pad_sequence(sentences,batch_first=True,padding_value=0)
  tags_padded=pad_sequence(tags,batch_first=True,padding_value=0)
  return sentences_padded,tags_padded
train_dataset=NERDataset(train_data,word_to_ix,tag_to_ix)
testa_dataset=NERDataset(testa_data,word_to_ix,tag_to_ix)
testb_dataset=NERDataset(testb_data,word_to_ix,tag_to_ix)
train_loader=DataLoader(train_dataset,batch_size=32,shuffle=True,collate_fn=collate_fn)
testa_loader=DataLoader(testa_dataset,batch_size=32,shuffle=False,collate_fn=collate_fn)
testb_loader=DataLoader(testb_dataset,batch_size=32,shuffle=False,collate_fn=collate_fn)

In [ ]:
class BiLSTMNER(nn.Module):
  def __init__(self,vocab_size,tagset_size,embedding_dim,hidden_dim):
    super(BiLSTMNER,self).__init__()
    self.embedding=nn.Embedding(vocab_size,embedding_dim)
    self.lstm=nn.LSTM(embedding_dim,hidden_dim//2,num_layers=1,bidirectional=True)
    self.hidden2tag=nn.Linear(hidden_dim,tagset_size)
  def forward(self,sentence):
    embeds=self.embedding(sentence)
    lstm_out,_=self.lstm(embeds.view(len(sentence),1,-1))
    tag_space=self.hidden2tag(lstm_out.view(len(sentence),-1))
    tag_scores=torch.log_softmax(tag_space,dim=1)
    return tag_scores

EMBEDDING_DIM = 64
HIDDEN_DIM = 128
VOCAB_SIZE = len(word_to_ix)
TAGSET_SIZE = len(tag_to_ix)





In [ ]:



param_grid = {
    'embedding_dim': [32, 64, 128],
    'hidden_dim': [64, 128, 256],
    'lr': [0.01, 0.1, 0.2],
    'num_epochs': [10]
}


all_combinations = list(itertools.product(*param_grid.values()))


def train_model(model, train_loader, loss_function, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for sentences, tags in tqdm(train_loader):
            model.zero_grad()
            tag_scores = model(sentences[0])
            loss = loss_function(tag_scores, tags[0])
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader)}")


def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for sentences, tags in test_loader:
            tag_scores = model(sentences[0])
            predicted_tags = torch.argmax(tag_scores, dim=1)
            correct += (predicted_tags == tags[0]).sum().item()
            total += len(tags[0])
    accuracy = correct / total
    return accuracy

best_accuracy = 0
best_params = {}

for params in all_combinations:
    embedding_dim, hidden_dim, lr, num_epochs = params


    model = BiLSTMNER(VOCAB_SIZE, TAGSET_SIZE, embedding_dim, hidden_dim)
    loss_function = nn.NLLLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr)


    train_model(model, train_loader, loss_function, optimizer, num_epochs)


    accuracy = evaluate_model(model, testa_loader)


    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_params = {'embedding_dim': embedding_dim, 'hidden_dim': hidden_dim, 'lr': lr, 'num_epochs': num_epochs}

print(f"Best Hyperparameters: {best_params}")
print(f"Best Accuracy on testa: {best_accuracy}")




best_model = BiLSTMNER(VOCAB_SIZE, TAGSET_SIZE, best_params['embedding_dim'], best_params['hidden_dim'])
loss_function = nn.NLLLoss()
optimizer = optim.SGD(best_model.parameters(), lr=best_params['lr'])


100%|██████████| 469/469 [00:04<00:00, 115.50it/s]


Epoch 1, Loss: 0.5213964443121638


100%|██████████| 469/469 [00:03<00:00, 123.78it/s]


Epoch 2, Loss: 0.2948697049290871


100%|██████████| 469/469 [00:03<00:00, 125.38it/s]


...

Epoch 8, Loss: 0.1693940535695332


100%|██████████| 469/469 [00:03<00:00, 123.29it/s]


Epoch 9, Loss: 0.16840424852359837


100%|██████████| 469/469 [00:04<00:00, 112.72it/s]


Epoch 10, Loss: 0.15773149248432747
Best Hyperparameters: {'embedding_dim': 128, 'hidden_dim': 128, 'lr': 0.2, 'num_epochs': 10}
Best Accuracy on testa: 0.9518156424581006


In [ ]:
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for sentences, tags in test_loader:
            tag_scores = model(sentences[0])
            predicted_tags = torch.argmax(tag_scores, dim=1)
            correct += (predicted_tags == tags[0]).sum().item()
            total += len(tags[0])
    accuracy = correct / total
    return accuracy
print(evaluate_model(model, testb_loader))

0.9426310583580614


In [ ]:
import torch
from transformers import BertTokenizerFast, BertForTokenClassification, Trainer, TrainingArguments
!pip install datasets
from datasets import Dataset
from sklearn.metrics import accuracy_score


tokenizer = BertTokenizerFast.from_pretrained("bert-base-cased")


model = BertForTokenClassification.from_pretrained("bert-base-cased", num_labels=9)


tag_to_ix = {
    'O': 0, 'B-ORG': 1, 'B-MISC': 2, 'B-PER': 3, 'I-PER': 4,
    'B-LOC': 5, 'I-ORG': 6, 'I-MISC': 7, 'I-LOC': 8
}


def tokenize_and_align_labels(data):
    tokenized_inputs = tokenizer(
        data["tokens"], truncation=True, is_split_into_words=True,
        padding='max_length', max_length=128
    )

    labels = []
    word_ids = tokenized_inputs.word_ids()
    previous_word_idx = None

    for word_idx in word_ids:
        if word_idx is None:
            labels.append(-100)
        elif word_idx != previous_word_idx:
            labels.append(tag_to_ix[data["ner_tags"][word_idx]])
        else:
            labels.append(tag_to_ix[data["ner_tags"][word_idx]])

        previous_word_idx = word_idx

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


train_dataset = Dataset.from_list([{'tokens': item[0], 'ner_tags': item[1]} for item in train_data]).map(tokenize_and_align_labels)
testa_dataset = Dataset.from_list([{'tokens': item[0], 'ner_tags': item[1]} for item in testa_data]).map(tokenize_and_align_labels)
testb_dataset = Dataset.from_list([{'tokens': item[0], 'ner_tags': item[1]} for item in testb_data]).map(tokenize_and_align_labels)


training_args = TrainingArguments(
    output_dir="./bert-ner",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
)


def compute_metrics(p):
    predictions, labels = p
    predictions = predictions.argmax(-1)


    true_labels = labels.flatten()
    pred_labels = predictions.flatten()


    valid_indices = true_labels != -100
    filtered_labels = true_labels[valid_indices]
    filtered_preds = pred_labels[valid_indices]

    accuracy = accuracy_score(filtered_labels, filtered_preds)
    return {"accuracy": accuracy}


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=testa_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


trainer.train()


test_results_testa = trainer.evaluate(testa_dataset)
test_results_testb = trainer.evaluate(testb_dataset)


print(f"Test results on TestA: {test_results_testa}")
print(f"Accuracy on TestA: {test_results_testa.get('eval_accuracy', 'Not Found')}")

print(f"Test results on TestB: {test_results_testb}")
print(f"Accuracy on TestB: {test_results_testb.get('eval_accuracy', 'Not Found')}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_syst

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: surampradeepkumar77 (surampradeepkumar77-sri-indu-college-of-engineering-tech) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.085100,0.064869,0.983373
2,0.032200,0.061961,0.985231
3,0.010000,0.065950,0.985793


Test results on TestA: {'eval_loss': 0.06196138262748718, 'eval_accuracy': 0.9852313987666417, 'eval_runtime': 25.8994, 'eval_samples_per_second': 133.825, 'eval_steps_per_second': 16.757, 'epoch': 3.0}
Accuracy on TestA: 0.9852313987666417
Test results on TestB: {'eval_loss': 0.16922901570796967, 'eval_accuracy': 0.9713934363231113, 'eval_runtime': 27.6324, 'eval_samples_per_second': 133.322, 'eval_steps_per_second': 16.683, 'epoch': 3.0}
Accuracy on TestB: 0.9713934363231113


In [ ]:
from transformers import BertTokenizerFast, BertForTokenClassification
import torch

model.save_pretrained("bert_ner_model")
tokenizer.save_pretrained("bert_ner_model")



model.eval()


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [ ]:
import torch
from transformers import BertTokenizerFast, BertForTokenClassification


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model = BertForTokenClassification.from_pretrained("bert_ner_model")
tokenizer = BertTokenizerFast.from_pretrained("bert_ner_model")


model.to(device)


def predict_ner(sentence):

    tokens = tokenizer(sentence, return_tensors="pt", is_split_into_words=True, truncation=True)


    tokens = {key: value.to(device) for key, value in tokens.items()}


    with torch.no_grad():
        outputs = model(**tokens)

    predictions = torch.argmax(outputs.logits, dim=-1).squeeze().tolist()


    tokenized_words = tokenizer.convert_ids_to_tokens(tokens["input_ids"].squeeze().tolist())


    ix_to_tag = {v: k for k, v in tag_to_ix.items()}


    print("\nNER Predictions:")
    for token, pred in zip(tokenized_words, predictions):
        if token not in ["[CLS]", "[SEP]", "[PAD]"]:
            print(f"{token} → {ix_to_tag[pred]}")


sentence = ["John", "lives", "in", "New", "York", "City", "and", "works", "at", "Google"]
predict_ner(sentence)



NER Predictions:
John → B-PER
lives → O
in → O
New → B-LOC
York → I-LOC
City → I-LOC
and → O
works → O
at → O
Google → B-ORG


In [ ]:
import torch

# Save the best BiLSTM model
torch.save(best_model.state_dict(), "bilstm_ner_model.pth")

# Load the model
loaded_model = BiLSTMNER(VOCAB_SIZE, TAGSET_SIZE, best_params['embedding_dim'], best_params['hidden_dim'])
loaded_model.load_state_dict(torch.load("bilstm_ner_model.pth"))
loaded_model.eval()


<ipython-input-12-6f289e6eed00>:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model.load_state_dict(torch.load("bilstm_ner_model.pth"))


BiLSTMNER(
  (embedding): Embedding(23625, 128)
  (lstm): LSTM(128, 64, bidirectional=True)
  (hidden2tag): Linear(in_features=128, out_features=9, bias=True)
)

In [ ]:
import torch

# Define the model architecture
class BiLSTMNER(nn.Module):
    def __init__(self, vocab_size, tagset_size, embedding_dim, hidden_dim):
        super(BiLSTMNER, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2, num_layers=1, bidirectional=True)
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.embedding(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = torch.log_softmax(tag_space, dim=1)
        return tag_scores

# Load the saved model
loaded_model = BiLSTMNER(VOCAB_SIZE, TAGSET_SIZE, best_params['embedding_dim'], best_params['hidden_dim'])
loaded_model.load_state_dict(torch.load("bilstm_ner_model.pth"))
loaded_model.eval()  # Set to evaluation mode


<ipython-input-13-e95205285989>:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model.load_state_dict(torch.load("bilstm_ner_model.pth"))


BiLSTMNER(
  (embedding): Embedding(23625, 128)
  (lstm): LSTM(128, 64, bidirectional=True)
  (hidden2tag): Linear(in_features=128, out_features=9, bias=True)
)

[('John', 'O'), ('lives', 'O'), ('in', 'B-ORG'), ('New', 'O'), ('York', 'B-ORG'), ('City', 'B-ORG')]


In [ ]:
from transformers import BertTokenizerFast, BertForTokenClassification
import torch

# Load the saved BERT model and tokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model = BertForTokenClassification.from_pretrained("bert_ner_model").to(device)
bert_tokenizer = BertTokenizerFast.from_pretrained("bert_ner_model")
bert_model.eval()


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [ ]:
def predict_bert(sentence, model, tokenizer, tag_to_ix):
    model.eval()
    tokens = tokenizer(sentence, return_tensors="pt", is_split_into_words=True, truncation=True)
    tokens = {key: value.to(device) for key, value in tokens.items()}

    with torch.no_grad():
        outputs = model(**tokens)

    predictions = torch.argmax(outputs.logits, dim=-1).squeeze().tolist()
    tokenized_words = tokenizer.convert_ids_to_tokens(tokens["input_ids"].squeeze().tolist())

    ix_to_tag = {v: k for k, v in tag_to_ix.items()}

    results = []
    for token, pred in zip(tokenized_words, predictions):
        if token not in ["[CLS]", "[SEP]", "[PAD]"]:
            results.append((token, ix_to_tag.get(pred, "O")))

    return results

sentence = ["John", "lives", "in", "New", "York", "City", "and", "works", "at", "Google"]
print(predict_bert(sentence, bert_model, bert_tokenizer, tag_to_ix))


[('John', 'B-PER'), ('lives', 'O'), ('in', 'O'), ('New', 'B-LOC'), ('York', 'I-LOC'), ('City', 'I-LOC'), ('and', 'O'), ('works', 'O'), ('at', 'O'), ('Google', 'B-ORG')]
